In [ ]:
from google.colab import files
uploaded = files.upload()  # Manually select `archive (16).zip` from your local machine

Saving archive (16).zip to archive (16) (1).zip


In [ ]:
import zipfile
zip_path = '/content/archive (16).zip'
extract_dir = '/content/dataset/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
import os
dataset_files = os.listdir(extract_dir)
print("Extracted files:", dataset_files)

Extracted files: ['diabetes_012_health_indicators_BRFSS2015.csv', 'diabetes_binary_health_indicators_BRFSS2015.csv', 'diabetes_binary_5050split_health_indicators_BRFSS2015.csv']


In [ ]:
import os
print("Files in dataset directory:", os.listdir(extract_dir))

Files in dataset directory: ['diabetes_012_health_indicators_BRFSS2015.csv', 'diabetes_binary_health_indicators_BRFSS2015.csv', 'diabetes_binary_5050split_health_indicators_BRFSS2015.csv']


In [ ]:
import pandas as pd
df = pd.read_csv(f'{extract_dir}diabetes_binary_health_indicators_BRFSS2015.csv')

# Example: Predict diabetes risk
from sklearn.ensemble import RandomForestClassifier
X = df.drop('Diabetes_binary', axis=1)
y = df['Diabetes_binary']
model = RandomForestClassifier().fit(X, y)

In [ ]:
!kaggle datasets download -d alexanderrich/ohiot1dm

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [ ]:
import numpy as np

def simulate_glucose(has_diabetes, bmi, exercise):
    base = 100 + 20*has_diabetes + 0.5*(bmi-25)
    return base + np.random.normal(0, 10 + 5*(1-exercise))

df['simulated_glucose'] = df.apply(
    lambda row: simulate_glucose(row['Diabetes_binary'], row['BMI'], row['PhysActivity']),
    axis=1
)

In [ ]:
# Install necessary libraries
!pip install ipywidgets --quiet
!jupyter nbextension enable --py widgetsnbextension --sys-prefix > /dev/null

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime, timedelta
import random

# Configure matplotlib for better rendering
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

# Pre-trained clustering model
class KMeansModel:
    def predict(self, X):
        """Predict cluster based on health metrics"""
        bmi, age, activity, _, _, _ = X[0]
        if bmi < 25 and activity > 1:
            return [0]  # Healthy
        elif bmi >= 30:
            return [2]  # Diabetic pattern
        elif age > 60:
            return [1]  # Prediabetic
        else:
            return [3]  # Active but high-risk

# Glucose simulation function
def simulate_glucose(user_type, days=1):
    """Generate realistic glucose patterns"""
    patterns = {
        0: {'base': 90, 'meal_effect': 20, 'variation': 8},    # Healthy
        1: {'base': 110, 'meal_effect': 35, 'variation': 12},  # Prediabetic
        2: {'base': 140, 'meal_effect': 50, 'variation': 18},  # Diabetic
        3: {'base': 100, 'meal_effect': 30, 'variation': 10}   # Active high-risk
    }
    params = patterns[user_type]

    # Create realistic time range starting from current time
    now = datetime.now()
    start_time = datetime(now.year, now.month, now.day, 6, 0)  # Start at 6 AM
    timestamps = [start_time + timedelta(hours=i) for i in range(24*days)]

    glucose = []
    for i, ts in enumerate(timestamps):
        val = params['base']

        # Add meal effects at breakfast (8 AM), lunch (1 PM), dinner (7 PM)
        if ts.hour in [8, 13, 19]:
            val += params['meal_effect'] * (1 + 0.1*np.random.randn())

        # Add circadian rhythm effects
        val += 5 * np.sin(2*np.pi*i/24)  # Daily cycle

        # Add random variation
        val += params['variation'] * np.random.randn()

        glucose.append(np.clip(val, 70, 250))

    return pd.DataFrame({'timestamp': timestamps, 'glucose': glucose})

# Custom CSS with blue, white, black and red theme
custom_css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap');

:root {
  --primary-blue: #1e40af;
  --light-blue: #3b82f6;
  --white: #ffffff;
  --black: #000000;
  --red: #dc2626;
  --light-gray: #f3f4f6;
  --dark-gray: #4b5563;
}

* {
  font-family: 'Poppins', sans-serif !important;
  color: var(--black);
}

body {
  background-color: var(--white);
}

.widget-label {
  font-weight: 500 !important;
  color: var(--black) !important;
}

.header-section {
  background: linear-gradient(135deg, var(--primary-blue), var(--light-blue));
  color: var(--white);
  padding: 25px 20px;
  border-radius: 12px;
  margin-bottom: 25px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.1);
  text-align: center;
}

.card {
  background: var(--white);
  border: 1px solid #e5e7eb;
  border-radius: 12px;
  padding: 20px;
  margin-bottom: 20px;
  box-shadow: 0 4px 6px rgba(0,0,0,0.05);
}

.results-card {
  background: var(--white);
  border: 1px solid #e5e7eb;
  border-radius: 12px;
  padding: 25px;
  margin: 20px 0;
  box-shadow: 0 6px 15px rgba(0,0,0,0.08);
}

.health-metric {
  display: flex;
  justify-content: space-between;
  background: var(--light-gray);
  padding: 12px 15px;
  border-radius: 8px;
  margin-bottom: 8px;
  font-weight: 500;
  border: 1px solid #e5e7eb;
}

.health-label {
  color: var(--black);
  font-weight: 500;
}

.health-value {
  color: var(--black); /* Changed to black as requested */
  font-weight: 600;
}

.recommendation-card {
  border-left: 4px solid var(--primary-blue);
  background: var(--light-gray);
  padding: 15px;
  border-radius: 0 8px 8px 0;
  margin: 15px 0;
}

.icon-header {
  display: flex;
  align-items: center;
  gap: 10px;
  margin-bottom: 15px;
}

.icon-header h3 {
  margin: 0;
  color: var(--primary-blue);
}

.btn-primary {
  background: var(--primary-blue) !important;
  border: none !important;
  font-weight: 500 !important;
  padding: 12px 28px !important;
  border-radius: 8px !important;
  transition: all 0.3s !important;
  color: var(--white) !important;
}

.btn-primary:hover {
  background: var(--light-blue) !important;
  transform: translateY(-2px);
  box-shadow: 0 4px 8px rgba(59, 130, 246, 0.3) !important;
}

.status-indicator {
  display: inline-block;
  width: 12px;
  height: 12px;
  border-radius: 50%;
  margin-right: 8px;
}

.status-0 { background: var(--primary-blue); }
.status-1 { background: var(--light-blue); }
.status-2 { background: var(--red); }
.status-3 { background: var(--dark-gray); }

.timeline {
  display: flex;
  justify-content: space-between;
  margin: 20px 0;
  padding: 15px;
  background: var(--light-gray);
  border-radius: 10px;
  border: 1px solid #e5e7eb;
}

.timeline-item {
  text-align: center;
  flex: 1;
}

.timeline-time {
  font-weight: 600;
  color: var(--primary-blue);
}

.timeline-value {
  font-size: 1.1rem;
  font-weight: 600;
}

.red-zone { color: var(--red); }
.blue-zone { color: var(--primary-blue); }

.alert-box {
  background-color: #fee2e2;
  border: 1px solid var(--red);
  border-radius: 8px;
  padding: 15px;
  margin: 15px 0;
}

.blue-highlight {
  color: var(--primary-blue);
  font-weight: 600;
}

.red-highlight {
  color: var(--red);
  font-weight: 600;
}

.profile-title {
  color: var(--primary-blue);
  margin-top: 0;
  border-bottom: 2px solid var(--primary-blue);
  padding-bottom: 10px;
}
</style>
"""

# Apply custom CSS
display(HTML(custom_css))

# Create widgets with enhanced styling
display(HTML("<div class='header-section'><h1>🩸 Diabetes Management Planner</h1><p>Get personalized meal and exercise recommendations based on your health profile</p></div>"))

# Interactive widgets
age = widgets.IntSlider(min=18, max=100, value=45, description="Age:", style={'description_width': '120px'})
bmi = widgets.FloatSlider(min=15, max=50, value=28, description="BMI:", style={'description_width': '120px'}, readout_format='.1f')
activity = widgets.Dropdown(
    options=[("Sedentary (little/no exercise)", 0),
             ("Light Activity (1-3 days/week)", 1),
             ("Active (3+ days/week)", 2)],
    value=1,
    description="Activity Level:",
    style={'description_width': '120px'}
)
diabetes_status = widgets.RadioButtons(
    options=[("Non-Diabetic", 0), ("Prediabetic", 1), ("Diabetic", 2)],
    value=0,
    description="Diabetes Status:",
    style={'description_width': '120px'}
)

# Display widgets in cards
display(HTML("<div class='card'><h3 class='profile-title'>👤 Your Health Profile</h3>"))
display(widgets.VBox([age, bmi, activity, diabetes_status]))

# Output area for results
output_area = widgets.Output()

# Initialize model
kmeans = KMeansModel()

# Generate Personalized Profile
def generate_profile(btn):
    with output_area:
        clear_output(wait=True)  # Clear previous results

        # Get user cluster prediction
        input_features = [[bmi.value, age.value, activity.value, 2, 2, 0]]
        user_cluster = kmeans.predict(input_features)[0]

        clusters = {
            0: "Healthy Pattern",
            1: "Prediabetic Pattern",
            2: "Diabetic Pattern",
            3: "Active High-Risk Pattern"
        }

        cluster_colors = {
            0: "#1e40af",  # Dark blue
            1: "#3b82f6",  # Light blue
            2: "#dc2626",  # Red
            3: "#4b5563"   # Dark gray (as black substitute)
        }

        # Display health profile - Age and BMI in black as requested
        display(HTML(f"""
        <div class='results-card'>
            <div class='icon-header'>
                <h3 style='color: #1e40af;'>Your Health Analysis</h3>
            </div>

            <div class='health-metric'>
                <span class='health-label'>Age</span>
                <span class='health-value'>{age.value}</span>
            </div>

            <div class='health-metric'>
                <span class='health-label'>BMI</span>
                <span class='health-value'>{bmi.value:.1f}</span>
            </div>

            <div class='health-metric'>
                <span class='health-label'>Activity Level</span>
                <span class='health-value' style='color: #1e40af;'>{activity.options[activity.index][0].split(' ')[0]}</span>
            </div>

            <div class='health-metric'>
                <span class='health-label'>Diabetes Status</span>
                <span class='health-value' style='color: #1e40af;'>{diabetes_status.options[diabetes_status.index][0]}</span>
            </div>

            <div class='health-metric' style='background-color: {cluster_colors[user_cluster]}10; border-left: 4px solid {cluster_colors[user_cluster]};'>
                <span class='health-label'>Health Pattern</span>
                <span class='health-value' style='color: {cluster_colors[user_cluster]}'>
                    <span class='status-indicator status-{user_cluster}'></span>
                    {clusters[user_cluster]}
                </span>
            </div>
        </div>
        """))

        # Simulate and plot glucose
        display(HTML("<div class='results-card'><div class='icon-header'><h3 style='color: #1e40af;'>📈 Glucose Pattern Simulation</h3></div>"))
        glucose_df = simulate_glucose(user_cluster)

        # Find key glucose metrics
        max_glucose = glucose_df['glucose'].max()
        min_glucose = glucose_df['glucose'].min()
        avg_glucose = glucose_df['glucose'].mean()

        # Find time of max glucose
        max_time = glucose_df.loc[glucose_df['glucose'].idxmax()]['timestamp'].strftime('%H:%M')

        fig, ax = plt.subplots()
        ax.plot(glucose_df['timestamp'], glucose_df['glucose'], color=cluster_colors[user_cluster], linewidth=3, alpha=0.8)

        # Add shaded regions for glucose zones
        ax.fill_between(glucose_df['timestamp'], 70, 180, color='#dbeafe', alpha=0.3)  # Light blue for target zone
        ax.fill_between(glucose_df['timestamp'], 180, 250, color='#fee2e2', alpha=0.3)  # Light red for hyper
        ax.fill_between(glucose_df['timestamp'], 0, 70, color='#fee2e2', alpha=0.3)     # Light red for hypo

        # Add threshold lines
        ax.axhline(180, color='#dc2626', linestyle='--', alpha=0.7, label='Hyperglycemia (>180)')
        ax.axhline(70, color='#dc2626', linestyle='--', alpha=0.7, label='Hypoglycemia (<70)')

        # Format x-axis for time
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        ax.set_xlabel('Time of Day', fontweight=500, color='black')
        ax.set_ylabel('Glucose (mg/dL)', fontweight=500, color='black')
        ax.set_title('Daily Glucose Pattern', fontsize=16, fontweight=600, color='black')
        ax.legend(loc='upper right')
        ax.grid(True, linestyle=':', alpha=0.5, color='#d1d5db')
        plt.xticks(rotation=45, color='black')
        plt.yticks(color='black')
        plt.tight_layout()
        plt.show()

        # Display key metrics
        display(HTML(f"""
        <div class="timeline">
            <div class="timeline-item">
                <div>Max Glucose</div>
                <div class="timeline-time">{max_time}</div>
                <div class="timeline-value {'red-zone' if max_glucose > 180 else 'blue-zone'}">{max_glucose:.0f} mg/dL</div>
            </div>
            <div class="timeline-item">
                <div>Min Glucose</div>
                <div class="timeline-value {'red-zone' if min_glucose < 70 else 'blue-zone'}">{min_glucose:.0f} mg/dL</div>
            </div>
            <div class="timeline-item">
                <div>Average</div>
                <div class="timeline-value {'red-zone' if avg_glucose > 180 else 'blue-zone'}">{avg_glucose:.0f} mg/dL</div>
            </div>
        </div>
        """))

        # Generate recommendations
        display(HTML("<div class='results-card'><div class='icon-header'><h3 style='color: #1e40af;'>📋 Personalized Recommendations</h3></div>"))

        meal_plan = {
            0: [
                "🥗 <b>Breakfast:</b> Greek yogurt with berries and nuts (30g carbs)",
                "🍗 <b>Lunch:</b> Grilled chicken salad with olive oil dressing (35g carbs)",
                "🐟 <b>Dinner:</b> Baked salmon with quinoa and steamed vegetables (40g carbs)"
            ],
            1: [
                "🥚 <b>Breakfast:</b> Veggie omelette with whole-grain toast (25g carbs)",
                "🥬 <b>Lunch:</b> Lentil soup with leafy green salad (30g carbs)",
                "🍠 <b>Dinner:</b> Turkey chili with sweet potato (35g carbs)"
            ],
            2: [
                "🥑 <b>Breakfast:</b> Avocado and eggs with low-carb bread (15g carbs)",
                "🥩 <b>Lunch:</b> Beef stir-fry with cauliflower rice (20g carbs)",
                "🥘 <b>Dinner:</b> Zucchini noodles with meatballs (25g carbs)"
            ],
            3: [
                "🍳 <b>Breakfast:</b> Mediterranean shakshuka (30g carbs)",
                "🥙 <b>Lunch:</b> Greek salad with grilled chicken (25g carbs)",
                "🍤 <b>Dinner:</b> Seafood paella with brown rice (40g carbs)"
            ]
        }

        exercise_plan = {
            0: [
                "🏃 <b>Morning:</b> 30 min brisk walk or light jogging",
                "💪 <b>Evening (3x/week):</b> Resistance training (20 min)"
            ],
            1: [
                "🚴 <b>Morning:</b> 45 min cycling or swimming",
                "🧘 <b>Evening:</b> 20 min yoga or stretching"
            ],
            2: [
                "🚶 <b>After meals:</b> 15 min walk after each meal",
                "🧘 <b>Daily:</b> 10 min light stretching"
            ],
            3: [
                "🚶 <b>Daily:</b> 10,000 steps throughout day",
                "🏋️ <b>3x/week):</b> Full-body strength training (30 min)"
            ]
        }

        # Display recommendations in cards
        display(HTML("""
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-top: 20px;">
            <div class="recommendation-card" style="border-left-color: #1e40af;">
                <h4 style="color: #1e40af; margin-top: 0;">🍽️ Meal Plan</h4>
                <ul style="padding-left: 20px;">
        """))

        for item in meal_plan[user_cluster]:
            display(HTML(f"<li>{item}</li>"))

        display(HTML("""
                </ul>
            </div>

            <div class="recommendation-card" style="border-left-color: #1e40af;">
                <h4 style="color: #1e40af; margin-top: 0;">💪 Exercise Plan</h4>
                <ul style="padding-left: 20px;">
        """))

        for item in exercise_plan[user_cluster]:
            display(HTML(f"<li>{item}</li>"))

        display(HTML("""
                </ul>
            </div>
        </div>

        <div class="recommendation-card" style="border-left-color: #1e40af; margin-top: 20px;">
            <h4 style="color: #1e40af; margin-top: 0;">⏰ Important Times</h4>
            <ul>
                <li><b>Check glucose:</b> Before meals and 2 hours after</li>
                <li><b>Best exercise times:</b> Morning (6-9 AM) or Evening (5-7 PM)</li>
                <li><b>Medication timing:</b> As prescribed by your doctor</li>
                <li><b>Key meals:</b> Consistent meal times help regulate glucose</li>
            </ul>
        </div>
        """))

        # Alert for high glucose levels
        if max_glucose > 180:
            display(HTML(f"""
            <div class="alert-box">
                <h4 style="color: #dc2626; margin-top: 0;">⚠️ Glucose Alert</h4>
                <p>Your simulated glucose reached <span class="red-highlight">{max_glucose:.0f} mg/dL</span> at {max_time}.
                This is above the recommended threshold of 180 mg/dL. Consider:</p>
                <ul>
                    <li>Reducing carbohydrate intake at meals</li>
                    <li>Taking a 15-minute walk after eating</li>
                    <li>Consulting your doctor about medication adjustments</li>
                </ul>
            </div>
            """))
        elif min_glucose < 70:
            display(HTML(f"""
            <div class="alert-box">
                <h4 style="color: #dc2626; margin-top: 0;">⚠️ Glucose Alert</h4>
                <p>Your simulated glucose dropped to <span class="red-highlight">{min_glucose:.0f} mg/dL</span>.
                This is below the safe threshold of 70 mg/dL. Consider:</p>
                <ul>
                    <li>Eating a small snack with 15g carbs when glucose is low</li>
                    <li>Carrying glucose tablets for emergencies</li>
                    <li>Reviewing medication timing with your doctor</li>
                </ul>
            </div>
            """))

        # Motivational quote
        quotes = [
            "Small daily improvements lead to stunning results over time.",
            "Your health is an investment, not an expense.",
            "Every step you take brings you closer to better health.",
            "Consistency is the key to managing diabetes successfully.",
            "Your journey to better health starts with one healthy choice."
        ]

        display(HTML(f"""
        <div style="background: linear-gradient(135deg, #1e40af, #3b82f6);
                    color: white;
                    padding: 15px;
                    border-radius: 10px;
                    margin-top: 20px;
                    text-align: center;">
            <h4 style="margin: 0;">💪 Today's Motivation</h4>
            <p style="font-style: italic; margin: 10px 0 0;">"{random.choice(quotes)}"</p>
        </div>
        """))

        # Disclaimer
        display(HTML("""
        <div style="font-size: 0.85em; color: #4b5563; margin-top: 25px; padding: 15px;
                    background: #f3f4f6; border-radius: 8px; border: 1px solid #e5e7eb;">
            <b>Disclaimer:</b> This tool provides general recommendations only. Always consult with your
            healthcare provider before making changes to your diet or exercise routine.
            Individual responses to foods and activities may vary.
        </div>
        """))

# Create and display submit button
submit_btn = widgets.Button(
    description="Generate My Plan",
    button_style='primary',
    icon='bolt',
    layout=widgets.Layout(width='220px', height='45px',
                          margin='20px auto',
                          font_weight='500',
                          font_size='16px')
)
submit_btn.on_click(generate_profile)

display(widgets.HBox([submit_btn], layout=widgets.Layout(justify_content='center')))
display(HTML("<div style='height: 20px'></div>"))  # Spacer

# Display area for results
display(output_area)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


Output()